In [ ]:
!git clone https://github.com/ashawkey/stable-dreamfusion.git

In [ ]:
 %pip install trimesh
 %pip install pythreejs

In [ ]:
!sudo apt-get install libeigen3-dev -y


In [ ]:
%cd stable-dreamfusion


In [ ]:
!git pull origin main

In [ ]:
%pip install  -r requirements.txt --force-reinstall
%pip install git+https://github.com/NVlabs/nvdiffrast/ 



In [ ]:
%pip install git+https://github.com/ashawkey/cubvh --force-reinstall

In [ ]:

# install CUDA extensions (takes about 8 minutes!)
%pip install ./raymarching --force-reinstall
%pip install ./shencoder --force-reinstall
%pip install ./freqencoder --force-reinstall
%pip install ./gridencoder --force-reinstall
#%pip install ./raymarching

In [ ]:
""" %cd pretrained/zero123
!cp /notebooks/threestudio/load/zero123/zero123-xl.ckpt  zero123-xl.ckpt
%cd ../../ """

In [ ]:
%cd pretrained/zero123
!rm zero123-xl.ckpt
""" !wget https://huggingface.co/cvlab/zero123-weights/resolve/main/105000.ckpt """
!wget https://zero123.cs.columbia.edu/assets/zero123-xl.ckpt
%cd ../../

In [ ]:
!mkdir pretrained/omnidata
%cd pretrained/omnidata
# assume gdown is installed
!gdown '18AAi0v9t3z-Mh4B8pGk-SdUegng88B3J&confirm=t' # omnidata_dpt_depth_v2.ckpt
!gdown '1gzMny6F3W7thYSM4Cr9gunxnPra1Tt6Z&confirm=t' # omnidata_dpt_normal_v2.ckpt
%cd ../../

In [ ]:
%cd pretrained/omnidata
!gdown '1gzMny6F3W7thYSM4Cr9gunxnPra1Tt6Z&confirm=t' # omnidata_dpt_normal_v2.ckpt
%cd ../../

In [ ]:
!git clone https://github.com/DrSleep/multi-task-refinenet


In [ ]:
%cd multi-task-refinenet
%pip install -r req.txt
%cd ../

In [ ]:
%cd stable-dreamfusion

In [ ]:
%env CUDA_VISIBLE_DEVICES=0
workspace ="werewolf_big"
image_path = f'/notebooks/threestudio/load/images/{workspace}_rgba.png'
image_path_without_rgba = image_path.replace('_rgba', '')


In [ ]:
!rm -rf ./{workspace}
!rm -rf ./{workspace}_dmtet

In [ ]:
%run preprocess_image.py {image_path_without_rgba}

In [ ]:
%cd multi-task-refinenet/src/notebooks
%matplotlib inline

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import sys
sys.path.append('../')
from models import net
import cv2
import torch
from torch.autograd import Variable
# Pre-processing and post-processing constants #
CMAP = np.load('../cmap_nyud.npy')
DEPTH_COEFF = 5000. # to convert into metres
HAS_CUDA = torch.cuda.is_available()
IMG_SCALE  = 1./255
IMG_MEAN = np.array([0.485, 0.456, 0.406]).reshape((1, 1, 3))
IMG_STD = np.array([0.229, 0.224, 0.225]).reshape((1, 1, 3))
MAX_DEPTH = 8.
MIN_DEPTH = 0.
NUM_CLASSES = 40
NUM_TASKS = 3 # segm + depth + normals
def prepare_img(img):
    return (img * IMG_SCALE - IMG_MEAN) / IMG_STD
model = net(num_classes=NUM_CLASSES, num_tasks=NUM_TASKS)
if HAS_CUDA:
    _ = model.cuda()
_ = model.eval()
ckpt = torch.load('../../weights/ExpNYUD_three.ckpt')
model.load_state_dict(ckpt['state_dict'])
# Figure 5-bottom row
img_path = image_path
img = np.array(Image.open(img_path).convert('RGB'))

with torch.no_grad():
    img_var = Variable(torch.from_numpy(prepare_img(img).transpose(2, 0, 1)[None]), requires_grad=False).float()
    if HAS_CUDA:
        img_var = img_var.cuda()
    segm, depth, norm = model(img_var)
    segm = cv2.resize(segm[0, :NUM_CLASSES].cpu().data.numpy().transpose(1, 2, 0),
                      img.shape[:2][::-1],
                      interpolation=cv2.INTER_CUBIC)
    depth = cv2.resize(depth[0, 0].cpu().data.numpy(),
                       img.shape[:2][::-1],
                       interpolation=cv2.INTER_CUBIC)
    norm = cv2.resize(norm[0].cpu().data.numpy().transpose(1, 2, 0),
                       img.shape[:2][::-1],
                       interpolation=cv2.INTER_CUBIC)
    segm = CMAP[segm.argmax(axis=2) + 1].astype(np.uint8)
    depth = np.abs(depth)
    out_norm = norm / np.linalg.norm(norm, axis=2, keepdims=True)
    ## xzy->RGB ##
    out_norm[:, :, 0] = ((out_norm[:, :, 0] + 1.) / 2.) * 255.
    out_norm[:, :, 1] = ((out_norm[:, :, 1] + 1.) / 2.) * 255.
    out_norm[:, :, 2] = ((1. - out_norm[:, :, 2]) / 2.) * 255.
    out_norm = out_norm.astype(np.uint8)

# mask


# Define the depth threshold
depth_threshold = 0.5
depth_img = cv2.imread(image_path.replace("rgba","depth"), cv2.IMREAD_GRAYSCALE)
# Create a boolean mask based on the depth threshold
# True for pixels in depth_img less than or equal to the depth_threshold, False otherwise
mask = depth_img > depth_threshold

# Apply the mask to the normal image
# First, we need to expand the dimensions of the mask to match the normal image
# Here we assume that the normal image is a 3-channel (RGB) image
mask_rgb = np.stack([mask]*3, axis=-1)

# Now, we apply the mask to the normal image. We could set the unmasked area to a specific color (like black), 
# or we could make the unmasked area transparent if the normal_img is a 4-channel (RGBA) image.
masked_img = np.where(mask_rgb, out_norm, 0)

# Save the image
image_dir, image_name = os.path.split(image_path_without_rgba)
image_basename, image_ext = os.path.splitext(image_name)
new_image_path = os.path.join(image_dir, f"{image_basename}_normal{image_ext}")

image = Image.fromarray(masked_img)
image.show()
image.save(new_image_path)
%cd ../../../

In [ ]:
batch_size = 13 # 24


In [ ]:

#%run preprocess_image.py front.png
## zero123 train
# pass in the processed <image>_rgba.png by  --default_polar 20 --image and do NOT pass in --text to enable zero-1-to-3 backend.--default_polar 0
%run main.py -O --image {image_path} --workspace {workspace}  --iters 5000 --test_interval 10 --batch_size {batch_size}

In [ ]:
%run main.py -O --batch_size {batch_size} --image {image_path} --test_interval 5 --workspace {workspace}_dmtet --dmtet   --init_with {workspace}/checkpoints/df.pth


In [ ]:
%run main.py -O --image {image_path} --workspace {workspace}_dmtet --dmtet --test --save_mesh 

In [ ]:
%cd /notebooks/stable-dreamfusion/{workspace}_dmtet/
!zip {workspace}.zip -r mesh
%cd ../

OTHER

In [ ]:
%run main.py -O  --image {image_path} --workspace {workspace}_dmtet_mesh  --dmtet --iters 5000 --init_with  {workspace}_dmtet/mesh/mesh.obj --lock_geo

In [ ]:
%run main.py -O --image {image_path} --workspace {workspace}_dmtet_mesh --dmtet --test --save_mesh

SIMPLE EXPORT

In [ ]:
%run main.py -O --image {image_path} --workspace {workspace} --test --save_mesh  

In [ ]:
!chmod +x scripts/run_image_procedure.sh

In [ ]:
import os
import subprocess

def run_image_procedure(GPU_ID, GUIDANCE_INTERVAL, DEFAULT_POLAR, PREFIX, PROMPT):
    EPOCHS1 = 100
    EPOCHS2 = 200
    EPOCHS3 = 300
    IMAGE = f"/notebooks/threestudio/load/images/{PREFIX}.png"
    IMAGE_RGBA = f"/notebooks/threestudio/load/images/{PREFIX}_rgba.png"
    WS_PH1 = f"trial_{PREFIX}-ph1"
    WS_PH2 = f"trial_{PREFIX}-ph2"
    WS_PH3 = f"trial_{PREFIX}-ph3"
    CKPT1 = f"{WS_PH1}/checkpoints/df_ep0{EPOCHS1}.pth"
    CKPT2 = f"{WS_PH2}/checkpoints/df_ep0{EPOCHS2}.pth"
    CKPT3 = f"{WS_PH3}/checkpoints/df_ep0{EPOCHS3}.pth"

    if not os.path.isfile(IMAGE_RGBA):
        %run preprocess_image.py {IMAGE}

    if not os.path.isfile(CKPT1):
        %run main.py -O --image {IMAGE_RGBA} --workspace {WS_PH1} --default_polar {DEFAULT_POLAR} --iters {EPOCHS1}00 --save_guidance --save_guidance_interval {GUIDANCE_INTERVAL} --batch_size 6 --test_interval 2 --h 96 --w 96 --zero123_grad_scale None --lambda_3d_normal_smooth 0 --dont_override_stuff --fovy_range 20 20 --guidance_scale 5 

    GUIDANCE_INTERVAL = 7
    if not os.path.isfile(CKPT2):
        %run main.py -O --image {IMAGE_RGBA} --workspace {WS_PH2} --text "{PROMPT}" --default_polar {DEFAULT_POLAR} --iters {EPOCHS2}00 --ckpt {CKPT1} --save_guidance --save_guidance_interval {GUIDANCE_INTERVAL} --h 128 --w 128 --albedo_iter_ratio 0.0 --t_range 0.2 0.6 --batch_size 4 --radius_range 2.2 2.6 --test_interval 2 --vram_O --guidance_scale 10 --jitter_pose --jitter_center 0.1 --jitter_target 0.1 --jitter_up 0.05 --known_view_noise_scale 0 --lambda_depth 0 --lr 0.003 --progressive_view --progressive_view_init_ratio 0.05 --known_view_interval 2 --dont_override_stuff --lambda_3d_normal_smooth 1 --textureless_ratio 0.0 --min_ambient_ratio 0.3 --exp_start_iter {EPOCHS1}00 --exp_end_iter {EPOCHS2}00

    if not os.path.isfile(CKPT3):
        %run main.py -O --image {IMAGE_RGBA} --workspace {WS_PH3} --text "{PROMPT}" --default_polar {DEFAULT_POLAR} --iters {EPOCHS3}00 --ckpt {CKPT2} --save_guidance --save_guidance_interval {GUIDANCE_INTERVAL} --h 512 --w 512 --albedo_iter_ratio 0.0 --t_range 0.0 0.5 --batch_size 1 --radius_range 3.2 3.6 --test_interval 2 --vram_O --guidance_scale 10 --jitter_pose --jitter_center 0.015 --jitter_target 0.015 --jitter_up 0.05 --known_view_noise_scale 0 --lambda_depth 0 --lr 0.003 --known_view_interval 2 --dont_override_stuff --lambda_3d_normal_smooth 0.5 --textureless_ratio 0.0 --min_ambient_ratio 0.3 --exp_start_iter {EPOCHS2}00 --exp_end_iter {EPOCHS3}00
run_image_procedure(0, 30, 70, "turtle_plant", "an 3D render of a pokemon ((mix between turtle and plant)) with ((red mark on feet)) and ((leaf crown)) on head")


In [ ]:

import os
def delete_workspace(PREFIX):
    WS_PH1 = f"trial_{PREFIX}-ph1"
    WS_PH2 = f"trial_{PREFIX}-ph2"
    WS_PH3 = f"trial_{PREFIX}-ph3"
    
    for workspace in [WS_PH1, WS_PH2, WS_PH3]:
        if os.path.isdir(workspace):
            os.system(f"rm -r {workspace}")
            print(f"Deleted workspace: {workspace}")
        else:
            print(f"No such workspace: {workspace}")
delete_workspace("turtle_plant")